In [32]:
import pprint 
import openpyxl
from openpyxl import load_workbook
import re

row_filter_mapping = {
    3: 'State ILP',
    4: 'ILP Webpage',
    5: 'ILP Contact Name + Email',
    6: 'ILP Policy',
    7: 'ILP Guide',
    8: 'State K12 Equity Plan',
    9: 'State ESSA Plan',
    10: 'ILP Equity Reference',
    11: 'ILP Funding',
    12: 'ILP PD/Training',
    13: 'ILP Standards',
    14: 'ILP Curriculum &/or Lessons',
    15: 'ILP Tech Platform',
    16: 'State SEL Webpage',
    17: 'SEL in ILP Reference',
    18: 'State Perkins V Plan',
    19: 'K12 CTE',
    20: 'K12 CTE, Career Pathways in ILP Reference',
    21: 'Dual Enrollment Webpage',
    22: 'IRC Webpage',
    23: 'WBL Webpage', 
    24: 'WBL Guide',
}

strings_to_be_removed = ['Filter1', 'Filter2', 'Filter3', 'Subfilter 1a', 
                         'Subfilter 1b', 'Subfilter 2a', 'Subfilter 2b', 'Subfilter 2c', 
                         'Subfilter 3a','Subfilter 3b','Subfilter 3c','Subfilter 3d','Subfilter 3e']

def remove_strings(list, strings_to_be_removed):
    for str in strings_to_be_removed:
        if str in list:
            list.remove(str)
    return list 

def read_all_data(workbook, row_filter_mapping, sheet_name):      
    sheet = workbook[sheet_name]    
    result = {} 
    row_num = 1
    for row in sheet.rows:
        cleanedup_row = []                
        if row_num >= 3:                    
            for cell in row: 
                link = []
                if cell.hyperlink:
                    parsed = cell.value.split(',')
                    if len(parsed) >= 2:
                        for str in parsed: 
                            res = re.findall('(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])', str.replace(' ', ''))
                            if len(res) > 0:
                                link.append(''.join(list(res[0])))
                        if len(link) > 0 :
                            cleanedup_row.extend(link)
                        else:
                            cleanedup_row.append(cell.hyperlink.target)
                    else:
                        cleanedup_row.append(cell.hyperlink.target)
                elif cell.value and cell.value != '':                             
                    #link = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', cell.value)
                    parsed = cell.value.split(',')                     
                    if len(parsed) >= 2: 
                        for str in parsed:                            
                            res = re.findall('(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])', str.replace(' ', ''))
                            if len(res) > 0:
                                link.append(''.join(list(res[0])))                            
                    else: 
                        res = re.findall('(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])', cell.value)                        
                        if len(res) > 0:
                            link.append(''.join(list(res[0])))
                    if len(link) > 0 :
                        cleanedup_row.extend(link)
                    else:
                        cleanedup_row.append(cell.value.replace('\n', ','))                        
            if row_num in row_filter_mapping:                
                result[row_filter_mapping[row_num]] = remove_strings(cleanedup_row[1:], strings_to_be_removed)
        row_num += 1             
    return result

output = {}
workbook = load_workbook(filename='careermap2.xlsx')
for sheet_name in workbook.sheetnames:
    if sheet_name != 'SPECs':       
        output[sheet_name] = read_all_data(workbook, row_filter_mapping, sheet_name)     


In [33]:
import pandas as pd 

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York ": "NY",
    "No Carolina": "NC",
    "No Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon ": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "So Carolina": "SC",
    "So Dakota": "SD",
    "Tennesee": "TN",
    "Texas RB": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia (DC)": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

scores = {}
df = pd.read_excel('scores.xlsx', skiprows=[0])
columns = df.columns.tolist()
del columns[0]

for index, row in df.iterrows():    
    scores[us_state_to_abbrev[row['State']]] = {}
    for score_name in columns:
        scores[us_state_to_abbrev[row['State']]][score_name] = row[score_name]

In [34]:
for state_name in output:
    for key in scores[state_name]:
        output[state_name][key] = scores[state_name][key]

print(output)        

{'WI': {'State ILP': ['ILP Name + (Abbreviation)', 'Academic & Career Planning (ACP)'], 'ILP Webpage': ['https://dpi.wi.gov/acp'], 'ILP Contact Name + Email': ['Karin Smith ,Karen.Smith@dpi.wi.gov '], 'ILP Policy': ['https://docs.legis.wisconsin.gov/code/admin_code/pi/26'], 'ILP Guide': ['https://dpi.wi.gov/sites/default/files/imce/acp/DPI%20Guide%202016%20FINAL%20web.pdf'], 'State K12 Equity Plan': ['LINK to State K12 Equity Plan or Webpage', 'https://dpi.wi.gov/statesupt/equity-council'], 'State ESSA Plan': ['https://dpi.wi.gov/sites/default/files/imce/esea/pdf/1%2012%2018%20WI%20Final%20ESSA%20Plan%20Submission.pdf'], 'ILP Equity Reference': ['https://dpi.wi.gov/acp/resources/equity'], 'ILP Funding': ['Dedicated funding reference LINK', 'https://dpi.wi.gov/acp/legal'], 'ILP PD/Training': ['ILP PD/Training LINK', 'https://dpi.wi.gov/acp/2021-22-career-readiness-community-practice'], 'ILP Standards': [], 'ILP Curriculum &/or Lessons': ['https://dpi.wi.gov/acp/resources/educator-tools'

In [35]:
import json 

states_coordinates = {}

f = open('us-state-boundaries.json')
data = json.load(f)

for rec in data:
    states_coordinates[rec['fields']['stusab']] = rec['fields']['st_asgeojson']['coordinates']   

In [37]:
geoJson = {'type': 'FeatureCollection', 'features': []}

for state_name in output:
    feature = {'type': 'Feature'}
    feature['properties'] = output[state_name]
    feature['properties']['state_name'] = state_name
    feature['geometry'] = {'type': 'Polygon', 'coordinates': states_coordinates[state_name]}
    geoJson['features'].append(feature)   


In [38]:
with open('states-careers3.json', 'w') as f:
    json.dump(geoJson, f)

In [14]:
import openpyxl
from openpyxl import load_workbook

workbook = load_workbook(filename='careermap.xlsx')
result = []
for sheet_name in workbook.sheetnames:
    row_num = 1
    #print(row_num)
    if sheet_name != 'SPECs': 
        sheet = workbook[sheet_name]        
        for row in sheet.rows:
            is_not_empty = False
            for cell in row:
                if cell.value:
                    if row_num > 24:
                        print(cell.value)
                    is_not_empty = True
                    break
            if is_not_empty:
                row_num += 1
        #print(row_num)
        if row_num > 25:
            result.append(sheet_name)
print(result)
            

25
State of Alabama Work-Based Learning State Report (plexamedia.com)
https://www.bcbe.org/site/handlers/filedownload.ashx?moduleinstanceid=843&dataid=21713&FileName=Revised_May_2012_Work_Based_Learning_Manual_with_form_permission.pdf
27
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
['AL']
